In [1]:
!pip install protobuf==3.20.3
!pip install --upgrade tokenizers
!pip install crewai crewai_tools langchain_community python-dotenv
!pip install exifread

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 12.3 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.25.5
    Uninstalling protobuf-4.25.5:
      Successfully uninstalled protobuf-4.25.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
grpcio-status 1.62.3 requires protobuf>=4.21.6, but you have protobuf 3.20.3 which is incompatible.


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
INFO: pip is looking at multiple versions of lancedb to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.0/192.0 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.0/468.0 kB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 60.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.8/131.8 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 64.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [2]:
from google.colab import userdata
import os

/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_config.py:345: UserWarning: Valid config keys have changed in V2:
* 'fields' has been removed
  warnings.warn(message, UserWarning)


In [3]:
input_dir = "original-pics"  # Replace with your input folder
downscaled_dir = "downscaled-pics"  # Replace with your output folder
selected_dir = "selected-pics"  # Replace with your output folder
example_web_files = "example-markdown"

#use the below directories for local runtime
# input_dir = "example-files/original-pics/"  # Replace with your input folder
# downscaled_dir = "example-files/downscaled-pics/"  # Replace with your output folder
# selected_dir = "example-files/selected-pics/"  # Replace with your output folder
# example_web_files = "example-files/example-markdown/"

In [4]:
from PIL import Image
import imghdr

def resize_images(input_folder, output_folder, max_dimension=1080):
    """
    Resizes images in the input folder, ensuring the largest dimension is max_dimension,
    maintaining aspect ratio, and saves them to the output folder, preserving date taken metadata.

    Args:
      input_folder: Path to the input folder containing images.
      output_folder: Path to the output folder where resized images will be saved.
      max_dimension: The maximum size for the largest dimension (width or height).
    """

    os.makedirs(output_folder, exist_ok=True)  # Create output folder if it doesn't exist

    for filename in os.listdir(input_folder):
        image_path = os.path.join(input_folder, filename)
        # Check if it's a file before checking image format
        if os.path.isfile(image_path) and imghdr.what(image_path) in ('png', 'jpeg', 'gif', 'bmp'):
            try:
                with Image.open(image_path) as img:
                    # exif_data = img.getexif()  # Get EXIF data
                    # date_taken = exif_data.get(36867) if exif_data else None  # Extract date taken

                    width, height = img.size

                    # Calculate new dimensions based on the largest dimension
                    if width > height:
                        new_width = max_dimension
                        new_height = int(height * (max_dimension / width))
                    else:
                        new_height = max_dimension
                        new_width = int(width * (max_dimension / height))

                    resized_img = img.resize((new_width, new_height), Image.Resampling.LANCZOS)  # Use LANCZOS for better quality

                    # Preserve EXIF data in the resized image
                    # if date_taken:
                    resized_img.info['exif'] = img.info['exif']

                    output_path = os.path.join(output_folder, filename)
                    resized_img.save(output_path, exif=resized_img.info.get('exif'))  # Save with EXIF data
                    print(f"Resized {filename} and saved to {output_folder}")
            except Exception as e:  # Handle potential errors
                print(f"Error processing {filename}: {e}")


resize_images(input_dir, downscaled_dir)

Resized P9021081.jpg and saved to downscaled-pics
Resized P9021062.jpg and saved to downscaled-pics
Resized P3040178.jpg and saved to downscaled-pics
Resized P5030087.jpg and saved to downscaled-pics
Resized P4170159.jpg and saved to downscaled-pics
Resized P5270907.jpg and saved to downscaled-pics
Resized P5260235.jpg and saved to downscaled-pics


In [5]:
import base64

def images_to_base64_dict(input_folder):
    """
    Converts images in the input folder to a dictionary with image names as keys
    and their base64 encoded values as values.

    Args:
      input_folder: Path to the input folder containing images.

    Returns:
      A dictionary where keys are image filenames and values are base64 encoded strings.
    """

    image_dict = {}

    for filename in os.listdir(input_folder):
        image_path = os.path.join(input_folder, filename)
        # Check if it's a file and a valid image format
        if os.path.isfile(image_path) and imghdr.what(image_path) in ('png', 'jpeg', 'gif', 'bmp'):
            try:
                with open(image_path, "rb") as image_file:
                    encoded_string = base64.b64encode(image_file.read()).decode("utf-8")
                    image_dict[filename] = encoded_string
            except Exception as e:
                print(f"Error processing {filename}: {e}")

    return image_dict

# Example usage:
downscaled_dir = "downscaled-pics"  # Replace with your input folder
image_dictionary = images_to_base64_dict(downscaled_dir)

# Print the dictionary (optional)
# print(image_dictionary)

In [6]:
agent_critic_background = """You are a highly respected art critic with 20 years of experience in the art market and 30 years
                          as a practicing artist. In your youth, you explored a wide range of artistic mediums,
                          from painting and sculpture to photography and performance art, giving you a deep understanding
                          of the creative process. Your passion for art has led you to travel the world,
                          immersing yourself in diverse cultures and artistic traditions. You possess a profound
                          knowledge of art history, from ancient cave paintings to contemporary installations,
                          and you are constantly seeking new knowledge through lectures, workshops, and exhibitions.
                          Your critical approach is characterized by a keen eye for visual storytelling,
                          an appreciation for the artist's process, and an openness to experimentation.
                          You are a master of articulating your thoughts in a clear and engaging manner,
                          providing insightful critiques that are both objective and constructive."""

agent_critic_task = """Please provide a detailed and objective analysis of the following image,
                        your analysis should include:
                      1. Objective Description:
                        - Provide a neutral and factual description of the image's subject matter, composition,
                            and key visual elements. Avoid subjective interpretations or emotional responses at this stage.
                        - Pay close attention to the use of color, line, form, texture, and space.
                      2. Critical Assessment:
                        - Your assessment will be based on how good you believe others will find the art work. If
                            you yourself like a picture but you think others don't find it striking or interesting
                            the image will get a lower score.
                        - Technical Execution: Analyze the artist's technical skill in handling the chosen medium
                            and their mastery of fundamental artistic principles (e.g., perspective, proportion,
                            light and shadow).
                        - Composition and Design: Evaluate the effectiveness of the composition in guiding the
                            viewer's eye and creating visual interest. Consider the use of balance, rhythm,
                            and focal points.
                        - Color Palette: Discuss the artist's use of color, noting the dominant hues, contrasts,
                            and their impact on the overall mood and atmosphere.
                        - Conceptual Exploration: If applicable, delve into the conceptual underpinnings of the work.
                            Identify any symbolism, metaphors, or narratives that contribute to its meaning.
                        - Originality and Innovation: Comment on the originality of the work and whether it
                            demonstrates a unique approach or expands upon existing artistic conventions.¨
                      3. Overall Impression:
                        - Summarize your overall impression of the artwork, highlighting its strengths and weaknesses.
                        - Offer constructive suggestions for improvement, if applicable.
                        - If an image is not good, boring, or poor, you are not afraid to say so.
                      4. Objective Quality Score:
                        - Assign a score between 1 and 10 (10 being the highest) based solely on your objective
                            assessment of the criteria above.
                        - It is crucial that your score is free from personal bias or preference. Focus on the
                            artwork's technical merits, compositional strength, and conceptual depth.
                        - If an image is not good, boring, or poor, you are not afraid to say so.
                        - Your score will be based on how good you believe others will find the art work. If
                            you yourself like a picture but you think others don't find it striking or interesting
                            the image will get a lower score.

                      Remember: Your role is to provide an objective and insightful critique, without aiming to please
                          your assessment is attachedd to your reputation, so you focus on being objective.
                          If an image is not good, boring, or poor, you are not afraid to say so."""

In [8]:
import litellm
import json
import time

critic_prompt = agent_critic_background + "\n\n" + agent_critic_task

def analyze_images(image_dictionary):
    """
    Analyzes images in a dictionary, sending them to Gemini for description,
    critical assessment, and quality score.

    Args:
        image_dictionary: A dictionary where keys are image names and values
                           are base64 encoded image strings.

    Returns:
        A dictionary containing analysis results for each image, following
        the defined schema.
    """

    os.environ['GEMINI_API_KEY'] = userdata.get("GOOGLE_API_KEY")

    results = {}

    for image_name, base64_image in image_dictionary.items():
        response = litellm.completion(
            # model="gemini/gemini-1.5-pro",
            model="gemini/gemini-1.5-flash",
            temperature=0.7,
            max_tokens=4096,
            messages=[
                {
                    "role": "user",
                    "content": [
                        {
                            "type": "text",
                            "text": critic_prompt
                        },
                        {
                            "type": "image_url",
                            "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"},
                        },
                    ],
                }
            ],
            # api_key=userdata.get("GOOGLE_API_KEY"),  # Replace with your API key source if needed
            response_format={
                "type": "json_object",
                "response_schema": {
                    "type": "object",
                    "properties": {
                        "description": {"type": "string"},
                        "critical_assessment": {"type": "string"},
                        "quality_score": {"type": "integer", "minimum": 1, "maximum": 10},
                    },
                    "required": ["description", "critical_assessment", "quality_score"],
                }
            }
        )

        try:
            analysis = json.loads(response.choices[0].message.content)
            results[image_name] = analysis
        except json.JSONDecodeError:
            print(f"Error decoding JSON response for image: {image_name}")

        time.sleep(1)  # Introduce a 2-second delay between API calls. not best practice but works for now

    return results

# Example usage:
# image_dictionary = {"image1.jpg": base64_image_string1, "image2.png": base64_image_string2, ...}
analysis_results = analyze_images(image_dictionary)
analysis_results_2 = analyze_images(image_dictionary)
analysis_results_3 = analyze_images(image_dictionary)
print(analysis_results)

{'P9021081.jpg': {'critical_assessment': "The photograph features a classic white convertible Peugeot 404 parked in front of the Panthéon in Paris. The composition is strong, using the architectural grandeur of the Panthéon to create a sense of scale and context for the car. The photographer skillfully utilizes natural light, creating highlights and shadows that accentuate the car's curves and the building's details. The color palette is understated yet elegant, with the creamy white of the car contrasting beautifully against the warm tones of the stone building and the bright blue sky.  Technically, the image is well-executed, with sharp focus and a balanced exposure. The perspective, from a low angle, adds dynamism and visual interest. The car is presented almost as a sculpture, showcased against a significant historical backdrop.  The conceptual exploration is subtle; it's a juxtaposition of classic design and timeless architecture, hinting at themes of elegance, history, and perhap

In [9]:
for key, value in analysis_results.items():
    # Accessing the 'quality_score' directly for averaging and updating
    quality_score = value.get('quality_score', 0)  # Using get() with default 0 for safety
    avg_quality_score = (quality_score + analysis_results_2[key].get('quality_score', 0) + analysis_results_3[key].get('quality_score', 0)) / 3.0

    # Updating the 'quality_score' in the original dictionary
    analysis_results[key]['quality_score'] = avg_quality_score
    # print(f"{key}    {quality_score}") # Printing the original quality score before update
    print(analysis_results[key]['quality_score'])

7.666666666666667
7.333333333333333
4.666666666666667
6.0
6.333333333333333
6.666666666666667
4.666666666666667


In [10]:
# prompt: save dictionary to txt file, with some formatting or line spaces that make it easy to read

import json
import os

def save_dict_to_txt(data, filename):
  """Saves a dictionary to a text file with formatted output.

  Args:
    data: The dictionary to save.
    filename: The name of the file to save to.
  """
  try:
    with open(filename, 'w') as file:
      for key, value in data.items():
        file.write(f"{key}:\n\n")
        if isinstance(value, dict):
          for k, v in value.items():
            file.write(f"  {k}: {v}\n\n")
        elif isinstance(value, list):
            file.write("  [\n\n")
            for item in value:
                file.write(f"    {item},\n\n")
            file.write("  ]\n\n")
        else:
          file.write(f"  {value}\n\n")
        file.write("\n\n")  # Add an extra newline for better readability

  except Exception as e:
      print(f"An error occurred: {e}")

# Example usage (assuming 'analysis_results' is defined as in your original code)
# Replace with your actual dictionary and file name
save_dict_to_txt(analysis_results, "image_analysis_results.txt")

In [11]:
# prompt: filter the dictionary analysis_results to only keep elements where quality_score is equal to 8 or higher

filtered_results = {key: value for key, value in analysis_results.items() if value.get('quality_score', 0) >= 7.3}
filtered_results

{'P9021081.jpg': {'critical_assessment': "The photograph features a classic white convertible Peugeot 404 parked in front of the Panthéon in Paris. The composition is strong, using the architectural grandeur of the Panthéon to create a sense of scale and context for the car. The photographer skillfully utilizes natural light, creating highlights and shadows that accentuate the car's curves and the building's details. The color palette is understated yet elegant, with the creamy white of the car contrasting beautifully against the warm tones of the stone building and the bright blue sky.  Technically, the image is well-executed, with sharp focus and a balanced exposure. The perspective, from a low angle, adds dynamism and visual interest. The car is presented almost as a sculpture, showcased against a significant historical backdrop.  The conceptual exploration is subtle; it's a juxtaposition of classic design and timeless architecture, hinting at themes of elegance, history, and perhap

In [ ]:
# class Image:
#     def __init__(self, filename, description, critical_assessment, quality_score, date_taken=None):
#         self.filename = filename
#         self.description = description
#         self.critical_assessment = critical_assessment
#         self.quality_score = quality_score
#         self.date_taken = date_taken

#     def get_date_taken(self, selected_dir):
#         """Extracts the date taken from the image file's metadata."""
#         try:
#             image_path = os.path.join(selected_dir, self.filename)  # Assume the image is in the selected_dir
#             with PILImage.open(image_path) as img:
#                 exif_data = img.getexif()
#                 date_taken = exif_data.get(36867) if exif_data else None
#                 self.date_taken = date_taken
#                 return date_taken
#         except Exception as e:
#             print(f"Error getting date taken for {self.filename}: {e}")
#             return None

In [12]:
from pydantic import BaseModel, Field, TypeAdapter
from typing import Dict, Optional, List, Set, Tuple
import os
import exifread

class ImageAnalysisResult(BaseModel):
    filename: str = Field(..., description="Filename of the image.")
    description: str = Field(..., description="Objective description of what is in the image.")
    assessment: str = Field(..., description="Qualitative assessment of the image.")
    quality_score: float = Field(..., ge=1, le=10, description="Quality score of the image based on assessment.")
    date_taken: str = Field(..., description="Date the image was originally captured.")
    title: str = Field(None, description="Title of the image.")
    summary: str = Field(None, description="Publication ready and web friendly summary of the image.")


def convert_to_objects(filtered_results, downscaled_dir):
    image_objects = []
    image_adapter = TypeAdapter(ImageAnalysisResult)  # Create a TypeAdapter
    for filename, analysis in filtered_results.items():
        image_path = os.path.join(downscaled_dir, filename)
        try:
            # Using exifread to get date taken:
            with open(image_path, 'rb') as f:
                tags = exifread.process_file(f)
                date_taken = tags.get('EXIF DateTimeOriginal', tags.get('Image DateTime'))

                # Extract only the date if date_taken is not None:
                if date_taken:
                    date_taken_str = str(date_taken)
                    date_only = date_taken_str.split()[0]  # Split by space and take the first part (date)
                else:
                    date_only = None  # Handle cases where date_taken is None

            # Create ImageAnalysisResult object using TypeAdapter
            image_object = image_adapter.validate_python({
                "filename": filename,
                "description": analysis.get('description', ''),
                "assessment": analysis.get('critical_assessment', ''),
                "quality_score": analysis.get('quality_score', 0),
                "date_taken": date_only  # Use the extracted date
            })
            image_objects.append(image_object)
        except FileNotFoundError:
            print(f"Warning: Image file not found: {image_path}")
        except Exception as e:
            print(f"Error processing {filename}: {e}")
    return image_objects

# Example usage (assuming filtered_results and downscaled_dir are defined)
image_objects = convert_to_objects(filtered_results, input_dir)  # Assuming 'input_dir' is your image directory

# Now you can access the attributes of the objects:
for obj in image_objects:
    print(f"Filename: {obj.filename}")
    print(f"Description: {obj.description}")
    print(f"Assessment: {obj.assessment}")
    print(f"Quality Score: {obj.quality_score}")
    print(f"Date Taken: {obj.date_taken}")
    print(f"Title: {obj.title}")  # Access the generated title
    print(f"Summary: {obj.summary}") # Access the generated summary
    print("-" * 20)

Filename: P9021081.jpg
Description: The image is a photograph depicting a classic white Peugeot 404 convertible parked in front of the Panthéon in Paris. The car is positioned slightly off-center, angled towards the viewer. The Panthéon, a large neoclassical building with a prominent dome, fills a significant portion of the background. The sky is a clear blue with a few fluffy white clouds. The photograph is taken from a low angle, emphasizing the car's size relative to the building. The lighting appears natural, likely taken outdoors on a sunny day. The colors are realistic, with the white of the car, the beige tones of the building, and the blue of the sky being the dominant hues. The texture of the car's paint and the stone of the building is visible. The composition is balanced, with the car acting as a focal point in the foreground, and the Panthéon providing a strong backdrop.
Assessment: The photograph features a classic white convertible Peugeot 404 parked in front of the Panth

In [26]:
# prompt: move all images remaining in analysis_results from downscaled_dir to selected_dir

import shutil

# Assuming analysis_results, downscaled_dir, and selected_dir are defined as in your provided code

os.makedirs(selected_dir, exist_ok=True)

for image_object in image_objects:
    source_path = os.path.join(downscaled_dir, image_object.filename)
    destination_path = os.path.join(selected_dir, image_object.filename)

    if os.path.exists(source_path):
        shutil.move(source_path, destination_path)
        print(f"Moved {image_object.filename} to {selected_dir}")
    else:
        print(f"Warning: {image_object.filename} not found in {downscaled_dir}")

In [ ]:
#add watermark to selected images

In [25]:

def organize_images(selected_dir):
    """
    Creates a new folder for each image in the selected directory,
    moves the image into its corresponding folder, renames it to "featured",
    and creates an index.md file.
    """
    for filename in os.listdir(selected_dir):
        name, ext = os.path.splitext(filename)
        if ext.lower() in ('.jpg', '.jpeg', '.png', '.gif', '.bmp'):
          try:
            # Create a new directory for the image
            new_dir = os.path.join(selected_dir, name)
            os.makedirs(new_dir, exist_ok=True)

            # Move and rename the image
            old_path = os.path.join(selected_dir, filename)
            new_path = os.path.join(new_dir, f"featured{ext}")
            os.rename(old_path, new_path)

            print(f"Moved and renamed '{filename}' to '{new_path}'")

            # Create index.md file
            index_md_path = os.path.join(new_dir, "index.md")
            with open(index_md_path, "w") as f:
                f.write(f"# {name}\n\n")  # Example content, you can customize this
                f.write("![featured](featured{ext})\n")

            print(f"Created 'index.md' in '{new_dir}'")

          except OSError as e:
              print(f"Error processing {filename}: {e}")

# Example usage:
organize_images(selected_dir)

In [15]:
from crewai import Agent, Task, Process, Crew, LLM
from crewai_tools import DirectoryReadTool, FileReadTool, FileWriterTool

llm_general = LLM(
    # model="gemini/gemini-1.5-pro",
    model="gemini/gemini-1.5-flash",
    temperature=0.4,
    max_tokens=4096,
    api_key=userdata.get('GOOGLE_API_KEY')
    # base_url="https://api.groq.com/openai/v1"
)

llm_developer = LLM(
    # model="gemini/gemini-1.5-pro",
    model="gemini/gemini-1.5-flash",
    temperature=0.2,
    max_tokens=4096,
    api_key=userdata.get('GOOGLE_API_KEY')
    # base_url="https://api.groq.com/openai/v1"
)

In [16]:
critic_full_agent = Agent(
    llm=llm_general,
    role="Art Critic",
    goal="Analyse visual art and write compelling SEO content",
    backstory=agent_critic_background,
    verbose=True,
    memory=True,
    # output_pydantic=ImageAnalysisResult
)

# Define the web developer agent with tools
web_dev_agent = Agent(
    llm=llm_developer,
    role="Web Developer",
    goal="Organize images and create website structure",
    backstory="""You are an experienced web developer proficient in HTML, CSS, Markdown, and JavaScript.
                You are tasked with organizing image files and working on markdown files.
                Use the tools provided to interact with the file system.""",
    verbose=True,
    memory=True
)

In [17]:
read_examples_task = Task(
    description=f"""Read all the example files from the directory '{example_web_files}'.
                    Store the content of these files in your memory for later use.""",
    expected_output="""Example files read and stored in memory.""",
    agent=web_dev_agent,
    # asynchronous=True,
    input_data={"example_files_dir": example_web_files},
    asynchronous=False,
    tools=[DirectoryReadTool(), FileReadTool()]  # Add the necessary tools
)

In [38]:
# class ImageAnalysisResults(BaseModel):  # Create a new model to wrap the list
#     results: List[ImageAnalysisResult]



# critic_read_examples_task = read_examples_task
# critic_read_examples_task.agent = critic_full_agent   #need this here
#solves the following error:
# ValidationError: 1 validation error for Crew
#   Sequential process error: Agent is missing in the task with the following description: Read all the example files from the directory '/content/example-markdown'.
#                     Store the content of these files in your memory for later use. [type=missing_agent_in_task, input_value={'agents': [Agent(role=Ar...e': True, 'cache': True}, input_type=dict]

# critic_read_examples_task = read_examples_task.copy_with(agent=critic_full_agent)

write_image_web_content = Task(
    description=f"""You are provided with a list of objects.
                   Each object contains the following information about an image:
                   - filename
                   - description
                   - assessment
                   - quality_score
                   - date_taken
                   - title (currently None)
                   - summary (currently None)

                   Read the content of this list and store it in your memory.
                   For each image object, generate:
                   1. A compelling title based on the description and assessment.
                   2. A compelling summary incorporating key features and artistic qualities.

                   You make sure the title and summary you write are in a similar style,
                   tone, voice, and length as in the example markdown files you have read.

                   The summary is describing the image only, with no reference to the artist himself,
                   as I am the artist and you are making sure the summary is well written on my behalf.
                   The summary should be written to be engaging, without seeming like it was written
                   by generative AI, i.e., use adjectives but do so without being excessive.

                   Make viewers and readers feel engaged and curious about the image, without tooting
                   my own horn too much. E.g., I would never describe my own composition as perfect,
                   I would be more subtle and descriptibe. This needs to be tasteful.

                   Store the generated title and summary within the image object,
                   by updating the 'title' and 'summary' attributes.

                   You do not need to open any files. The list of objects is provided as input data here:
                   '{image_objects}'""",
    expected_output="""List of ImageAnalysisResult objects read, titles and summaries generated, and stored within the objects.""",
    agent=critic_full_agent,
    asynchronous=False,
    input_data={
          "image_objects": image_objects  # Pass the list of ImageAnalysisResult objects
    },
    context=[read_examples_task],
    # output_pydantic=ImageAnalysisResult #implement later, similar errors https://github.com/crewAIInc/crewAI/discussions/1436
)

review_image_web_content = Task(
    description=f"""You perform a review of the each image title and image summary.
                    You make sure the title and summary are written in a similar style,
                   tone, voice, and length as in the example markdown files you have read.

                   The summary is describing the image only, with no reference to the artist himself,
                   as I am the artist and you are making sure the summary is well written on my behalf.
                   The summary should be written to be engaging, without seeming like it was written
                   by generative AI, i.e., use adjectives but do so without being excessive.

                   Make viewers and readers feel engaged and curious about the image, without tooting
                   my own horn too much. E.g., I would never describe my own composition as perfect,
                   I would be more subtle and descriptibe. This needs to be tasteful.

                   You make any changes you deem necessary to improve the text.""",
    expected_output="""List of ImageAnalysisResult objects read, titles and summaries generated, and stored within the objects.""",
    agent=critic_full_agent,
    asynchronous=False,
    # input_data={
    #       "image_objects": image_objects  # Pass the list of ImageAnalysisResult objects
    # },
    context=[write_image_web_content],
    # output_pydantic=ImageAnalysisResult #implement later, similar errors https://github.com/crewAIInc/crewAI/discussions/1436
)


In [39]:

process_all_subfolders_task = Task(
    description=f"""Navigate to the directory '{selected_dir}'.
                   For each subfolder in this directory:
                       1. Get the name of the current subfolder.
                       2. Find the corresponding image object in the '{image_objects}' list
                          by matching the subfolder name with the object's 'filename' attribute (without extension).
                       3. Open the file named 'index.md' in the subfolder. If it doesn't exist, create it.
                       4. Based on the example web markdown files you read earlier and the image object's data,
                          populate the 'index.md' file using the following attributes from the '{image_objects}':
                           - Title: Use the 'title' attribute from the image object.
                           - Date: Use the 'date_taken' attribute from the image object.
                           - Image Summary: Use the 'summary' attribute from the image object.
                           - Image: Ensure the image path in the markdown file is correct (e.g., '![](featured.jpg)').
                       5. Repeat steps 1-4 for all subfolders in '{selected_dir}'.
                   Use DirectoryReadTool to list files, FileReadTool to read files, and FileWriterTool to update/create the 'index.md' files.
                   It is important that you do not rewrite the image summary or the image title. You should use the
                   summary and the title you received from the Art Critic Agent""",
    expected_output=f"""'index.md' files updated/created in all subfolders of '{selected_dir}' with content from image objects.
                    the file must include the attributes:
                    "title: "
                    "authors:
                      -admin"
                    "tags:
                      - Photography"
                    "image:
                      placement: 2
                      preview_only: false"
                    """,
    agent=web_dev_agent,
    context=[read_examples_task, review_image_web_content],
    tools=[DirectoryReadTool(), FileReadTool(), FileWriterTool()],  # Add the necessary tools
    asynchronous=False,
    input_data={
        "selected_dir": selected_dir,
        # "image_objects": image_objects,
        "example_files_dir": example_web_files
    }
)

# review_index_structure = Task(
#     description=f"""You open the folder '{selected_dir}'.
#                     In this folder there are multiple sub folders, each named after an image.
#                     You iterate through each sub folder. You read the index.md files in each subfolder
#                     and compare the file to the example markdown files you previously read
#                     in the task 'read_examples_task'.

#                     You verify that the technical structure of the index.md files is correct,
#                     and the technical structure is identical to the example index markdown files.

#                     This includes verifying the correct variables, date structure, and similar.
#                     You DO NOT change any of the content itself, which is the title and the image description.
#                     """,
#     expected_output=f"""'index.md' files updated in all subfolders of '{selected_dir}' with content from image objects.
#                     The title and the image description/summary will NOT be changed by you.
#                     The file must include the attributes:
#                     "title: "
#                     "authors:
#                       -admin"
#                     "tags:
#                       - Photography"
#                     "image:
#                       placement: 2
#                       preview_only: false"
#                     """,
#     agent=web_dev_agent,
#     asynchronous=False,
#     context=[read_examples_task, process_all_subfolders_task],
#     tools=[DirectoryReadTool(), FileReadTool(), FileWriterTool()],
#     input_data={
#         "selected_dir": selected_dir
#     }
# )

In [40]:
full_crew = Crew(
    agents=[critic_full_agent,
            web_dev_agent
            ],
    tasks=[
           read_examples_task,
           write_image_web_content,
           review_image_web_content,
           process_all_subfolders_task,
          #  review_index_structure
           ],
    process=Process.sequential,
    verbose=True,
    # memory=True,
    cache = True,
)

results = full_crew.kickoff()
print(results)

# Agent: Web Developer
## Task: Read all the example files from the directory 'example-markdown'.
                    Store the content of these files in your memory for later use.


# Agent: Web Developer
## Thought: Thought:I need to list all files in the 'example-markdown' directory first to know which files to read.
## Using tool: List files in directory
## Tool Input: 
"{\"directory\": \"example-markdown\"}"
## Tool Output: 
File paths: 
-example-markdown/index1.md
- example-markdown/index2.md
- example-markdown/index3.md


# Agent: Web Developer
## Thought: Thought:Now I need to read the content of each file.
## Using tool: Read a file's content
## Tool Input: 
"{\"file_path\": \"example-markdown/index1.md\"}"
## Tool Output: 
---
title: 'Roadrunner in Big Bend National Park'
authors:
- admin
tags:
- Photography
date: "2023-04-06T00:00:00Z"
featured: false
draft: false

image:
  placement: 2
  focal_point: ""
  preview_only: false
---

A Greater Roadrunner stands alert on a rocky

In [42]:
# test_crew = Crew(
#     agents=[critic_full_agent,
#             # web_dev_agent
#             ],
#     tasks=[
#            critic_read_examples_task,
#            write_image_web_content,
#            review_image_web_content
#           #  dev_read_examples_task,
#           #  process_all_subfolders_task
#            ],
#     process=Process.sequential,
#     verbose=True,
#     # memory=True,
#     cache = True,
# )

# results = test_crew.kickoff()
# print(results)

In [43]:
# Now you can access the attributes of the objects:
# for obj in image_objects:
#     print(f"Filename: {obj.filename}")
#     print(f"Description: {obj.description}")
#     print(f"Assessment: {obj.assessment}")
#     print(f"Quality Score: {obj.quality_score}")
#     print(f"Date Taken: {obj.date_taken}")
#     print(obj.title)  # Access the generated title
#     print(obj.summary) # Access the generated summary
#     print("-" * 20)